### Neural Machine Translation with Keras

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
# Building a english to french translator

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_30/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
lines= pd.read_table(r'/kaggle/input/fra-txt/fra.txt', names=['eng', 'fr'])

In [4]:
lines.shape

(149861, 2)

In [5]:
lines = lines[0:20000]

In [6]:
lines.shape

(20000, 2)

In [7]:
lines.head(10)

,eng,fr
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !
5,Help!,À l'aide !
6,Jump.,Saute.
7,Stop!,Ça suffit !
8,Stop!,Stop !
9,Stop!,Arrête-toi !


#### Data Cleanup

In [8]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fr=lines.fr.apply(lambda x: x.lower())

In [9]:

lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.fr=lines.fr.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [10]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

The translate() method returns a string where some specified characters are replaced with the character described in a dictionary(str.maketrans), or in a mapping table.

In [11]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))

In [12]:
lines.head(10)

,eng,fr
0,go,va
1,run,cours
2,run,courez
3,wow,ça alors
4,fire,au feu
5,help,à laide
6,jump,saute
7,stop,ça suffit
8,stop,stop
9,stop,arrêtetoi


#### Generate  data

In [13]:
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
len(lines.eng)

20000

l'ensemble des mots uniques de chaque vocabulaire

In [15]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)



all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [16]:
len(all_eng_words), len(all_french_words)

(3447, 7194)

calcul de chaque sequence par mots

In [17]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))

print(lines.eng[0].split())
np.max(lenght_list)

['START_', 'go', '_END']


7

In [32]:
lenght_list=[]
for l in lines.fr:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

12

mots + index

In [19]:
input_words = sorted(list(all_french_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_french_words)
num_decoder_tokens = len(all_eng_words)
# del all_eng_words, all_french_words

In [20]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

matrice pour input de encoder et decoder et matrice de output de decoder

In [37]:
encoder_input_data = np.zeros(
    (len(lines.fr), 12),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.eng), 7),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.eng), 7, num_decoder_tokens),
    dtype='float32')

In [38]:
decoder_target_data.shape

(20000, 7, 3447)

In [39]:
encoder_input_data.shape

(20000, 12)

In [40]:
decoder_input_data.shape

(20000, 7)

In [41]:
input_token_index['non'] #was go

4349

In [42]:
for i, (input_text, target_text) in enumerate(zip(lines.fr, lines.eng)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character. start word = 1
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [43]:
decoder_target_data.shape

(20000, 7, 3447)

### Build keras encoder-decoder model

In [44]:
embedding_size =50

In [45]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

#### Encoder model

In [46]:
encoder_inputs = Input(shape=(None,))#préparer l'input shape
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)#préparer l'embedding
encoder = LSTM(50, return_state=True)# créer l'LSTM de l'encodeur / step size par defaut= nbre des tokens (50)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)# ce petit modèle d'encodage reçoit la séquence et génère (h, c) qui sont communiqué,
# par la suite au décodeur.
encoder_model.summary()


#.add = automatically gives input n output however here were precising the input and output of each layer

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 50)       │       359,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 50), (None,    │        20,200 │
│                                 │ 50), (None, 50)]       │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 379,900 (1.45 MB)

 Trainable params: 379,900 (1.45 MB)

 Non-trainable params: 0 (0.00 B)

#### Decoder model

In [47]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,)) #préparer l'input shape

dex=  Embedding(num_decoder_tokens, embedding_size) #préparer l'embedding

final_dex= dex(decoder_inputs)# fournir la data à l'embedding


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)# créer l'LSTM decodeurcodeur

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)# lui donner l'mbedding de la data et l'output de l'encodeur (h+c)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')# prédire le token

decoder_outputs = decoder_dense(decoder_outputs)#l'appliquer

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)# on a deux inputs: 1 pour l'encodeur et un autre pour le decodeur ,
#pour le décodeur initialement sera le (h,c) de l'encodeur et puis ça sera le h,c de la cellule précédente.

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [48]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 50)  │    359,700 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 50)  │    172,350 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 50),      │     20,200 │ embedding[0][0]   │
│                     │ (None, 50),       │            │                   │
│                     │ (None, 50)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     20,200 │ embedding_1[0][0… │
│                     │ 50), (None, 50),  │            │ lstm[0][1],       │
│                     │ (None, 50)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │    175,797 │ lstm_1[0][0]      │
│                     │ 3447)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 748,247 (2.85 MB)

 Trainable params: 748,247 (2.85 MB)

 Non-trainable params: 0 (0.00 B)

#### Fit the model

In [49]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=1,
          validation_split=0.1)

141/141 ━━━━━━━━━━━━━━━━━━━━ 31s 192ms/step - acc: 0.1347 - loss: 4.2263 - val_acc: 0.1429 - val_loss: 3.5731


In [50]:
model.save_weights('nmt.weights.h5')

In [51]:
model.load_weights('nmt.weights.h5')

In [52]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 50)       │       359,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 50), (None,    │        20,200 │
│                                 │ 50), (None, 50)]       │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 379,900 (1.45 MB)

 Trainable params: 379,900 (1.45 MB)

 Non-trainable params: 0 (0.00 B)

#### Create sampling model (inférence)

In [58]:
#input for the decoder
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

#output for the decoder
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

#dense layer for prediction
decoder_outputs2 = decoder_dense(decoder_outputs2)

#architecture of the decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

#### Function to generate sequences

In [59]:
target_token_index['_END']

2

In [64]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    states_value[1].shape
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    print('target_seq')
    print(target_seq)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        print(output_tokens.shape)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 7):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Look at the some translations

In [65]:
encoder_input_data[690: 690 + 1]

array([[7.021e+03, 1.000e+00, 7.037e+03, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00]],
      dtype=float32)

In [66]:
for seq_index in [690]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
target_seq
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(1, 1, 3447)
1509
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
(1, 1, 3447)
2
-
Input sentence: 690    START_ it failed _END
Name: eng, dtype: object
Decoded sentence:  i _END
